In [89]:
import numpy as np
from PIL import Image
import cv2 as cv
from matplotlib import pyplot as plt
from scipy.linalg import hessenberg

In [90]:
def eigenQR(A):
    temp,_ = hessenberg(A, True)
    #temp = np.copy(A)
    row = temp.shape[0]
    QQ = np.eye(row)

    for i in range(100000):
        c = temp[row-1, row-1]
        cI = c * np.eye(row)
        Q, R = np.linalg.qr(np.subtract(temp, cI))
        temp = np.add(R @ Q, cI)

        QQ = QQ @ Q
    
    return np.diag(temp), QQ

In [91]:
def methodSVD(matrix):
    row, col = matrix.shape
    
    eigAtA, rightSingular= eigenQR(np.transpose(matrix) @ matrix)
    Vt = np.transpose(rightSingular)
    rank = np.count_nonzero(eigAtA)

    #Sigma = np.zeros((row, col))
    SigmaInv = np.zeros((row, col))
    eigSort = np.sort(eigAtA)[::-1]
    #eigAtA = eigAtA[::-1].sort()
    for i in range(len(eigSort)):
        #Sigma[i][i]= np.sqrt(eigSort[i])
        SigmaInv[i][i] = 1/np.sqrt(eigSort[i])

    U = matrix @ Vt @ SigmaInv
    
    return U, eigSort, Vt

In [92]:
a = np.array([[3, -2, 0], [-2, 3, 0], [0, 0, 5]]).astype(float)

a, b, c = methodSVD(a)
print(a)
print(b)
print(c)

[[-0.14142136 -0.70710678  0.        ]
 [-0.14142136  0.70710678  0.        ]
 [ 0.          0.          5.        ]]
[25. 25.  1.]
[[-0.70710678 -0.70710678  0.        ]
 [-0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]


In [93]:
def load_image():
    img_name = input("Input image name: ")
    image = Image.open(img_name)

    img_arr = np.asfarray(image)
    print(img_arr)
    print(img_arr.shape)

    return img_arr

In [96]:
def svd_compress_gs(img, k):
	"""Given a matrix representing a grayscale image, compress 
	it by taking the largest k elements from its singular values"""
	U, singular_vals, V = methodSVD(img)
	rank = len(singular_vals)
	print("Image rank %r" % rank)
	if k > rank:
		print("k is larger than rank of image %r" % rank)
		return img
	# take columns less than k from U
	U_p = U[:,:k]
	# take rows less than k from V
	V_p = V[:k,:]
	# build the new S matrix with top k diagnal elements
	S_p = np.zeros((k, k), img.dtype)
	for i in range(k):
		S_p[i][i] = singular_vals[i]
	print("U_p shape {0}, S_p shape {1}, V_p shape {2}".format(
		U_p.shape, S_p.shape, V_p.shape))
	compressed = np.dot(np.dot(U_p, S_p), V_p)
	ss = ssim(img, compressed,
		dynamic_range=compressed.max()-compressed.min())
	print("Strucural similarity: %r" % ss)
	return compressed

In [97]:
comp = svd_compress_gs(load_image(), 10)
plt.imshow(comp, interpolation='nearest')
plt.show()

[[250. 220. 222. ... 226. 222. 220.]
 [222.  81.  90. ... 107.  87.  79.]
 [222.  88.  96. ... 111.  93.  85.]
 ...
 [186.   5.   9. ...  35.  35.  35.]
 [186.   5.   9. ...  36.  37.  37.]
 [186.   5.   9. ...  37.  37.  37.]]
(850, 850)
Image rank 850
U_p shape (850, 10), S_p shape (10, 10), V_p shape (10, 850)


NameError: name 'ssim' is not defined